# Modelos 4
Support Vector Machine

Nome: Joaquim Junior Ferola Fonseca e Matheus Pereira Velloso da Silveira

### 1. Introdução
Nesse notebook, será visto o funcionamento e uso do consagrado modelo de Machine Learning, Support Vector Machine.

### 2. Teoria
O SVC (Support Vector Machine) tem como objetivo basicamente traçar áreas de classificação. Ou seja, suponha dois atributos e as classificações. O SVC poderia traçar retas onde tem a maior probabilidade de ser a fronteira entre as classes. Veja isso visualmente abaixo:
<div align="center">
    <img src="https://media.geeksforgeeks.org/wp-content/uploads/20200605170732/linearsep.png" alt="Fern vs Ehre" width="600">
</div>

Podemos continuar pensando em 2 atributos, para simplificar a visualização. Para mais dimensões é análogo, só não é fácil representar em uma figura. Essas divisões de regiões podem se dar de diferentes formas: linear (quando as divisões são dadas por retas, como no exemplo A), polinomial (podendo ser segmentos dados por funções polinomiais), radial basis function (rbf)(as curvas são formadas por curvas elípticas) e sigmoid (parecido com redes neurais em que as curvas são sigmoides).
<div align="center">
    <img src="https://editor.analyticsvidhya.com/uploads/28931svm32.png" alt="Fern vs Ehre" width="600">
</div>

Vamos agora aprofundar para a mais simples, linear kernel.

O objetivo é criar retas que separem os dados com a maior distância entre a reta (que é denominada hyperplane) e o dado mais próximo.
<div align="center">
    <img src="https://media.geeksforgeeks.org/wp-content/uploads/20250805115844223142/SVM.webp" alt="Fern vs Ehre" width="600">
</div>


A reta do hiperplano é dada por: $$\omega^\tau x + b = 0$$
Em que: 

x é um ponto de elemento com suas coordenadas sendo os atributos representados no plano.

w é o vetor normal, ou seja, sempre perpendicular ao hiperplano.

b é chamado bias, representa a distância do hiperplano à origem

Nessa equação, o primeiro termo é um produto escalar entre os vetores w e x. Podemos interpretar essa equação como uma reta no plano. Assim, podemos também calcular a distância a um ponto:
$$
d_i = \frac{\omega^\tau x + b}{||\omega||}
$$
Agora, podemos definir pra qual lado do hiperplano está o ponto. Para isso:
$$
\hat{y} = \begin{cases} 1 & \text{se } \omega^\tau x + b \geq 0 \\
0 & \text{se } \omega^\tau x + b < 0
\end{cases}
$$

Assim, como o objetivo, como foi discutido acima é maximizar as distâncias. Assim, o objetivo do SVM é calcular:
$$
minimize ||\omega||^2 = \omega^\tau\omega
$$
Idealmente teríamos:
$$
y_i(\omega^\tau x + b) \geq 0
$$
Mas como não teremos necessariamente todos os elementos nas áreas certas, devemos adicionar uma penalidada associada a esses. Assim, usamos:
$$
\zeta_i \space \text{a distância ao ponto i, balanceando a equação que devemos minimizar}
$$
E a constante C é o peso dado a essas distâncias errôneas. Assim, o SVM basicamente visa minimizar a equação:
$$
minimize \frac{1}{2}||\omega||^2 + C \sum_{i=1}^{n} \zeta_i
$$

### 3. Uso

#### 3.1 Importação e tratamento do Dataframe
Usarei o student_exam_scores.csv, disponível gratuitamente na plataforma Kaggle.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

Tratando os dados:

In [2]:
df = pd.read_csv("student_exam_scores.csv")
df = df.iloc[0:, 1:]
df = df.dropna()

for i in range(200):
    if (df["exam_score"][i]) > 40:
        df.loc[i, "exam_score"] = "A"
    elif (df["exam_score"][i]) > 30:
        df.loc[i, "exam_score"] = "B"
    elif (df["exam_score"][i]) > 20:
        df.loc[i, "exam_score"] = "C"
    else:
        df.loc[i, "exam_score"] = "D"
df['exam_score'] = df['exam_score'].astype(str)
df

C:\Users\matheus25022\AppData\Local\Temp\ipykernel_15652\201314214.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'B' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, "exam_score"] = "B"


,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score
0,8.0,8.8,72.1,45,B
1,1.3,8.6,60.7,55,C
2,4.0,8.2,73.7,86,B
3,3.5,4.8,95.1,66,B
4,9.1,6.4,89.8,71,A
...,...,...,...,...,...
195,10.5,5.4,94.0,87,A
196,7.1,6.1,85.1,92,A
197,1.6,6.9,63.8,76,C
198,12.0,7.3,50.5,58,A


Divisão treino teste:

In [3]:
X = df.drop("exam_score", axis=1)
y = df["exam_score"]
SEMENTE = 27
#Divisão treino teste
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.1, random_state=SEMENTE)

#### 3.2 Baseline

Usemos agora um modelo baseline. Usarei o DummyClassifier com a estratégia most_frequent, ou seja, que só retorna o target mais frequente do dataframe. Isso é importante, pois se trata de um modelo que não aprende, ou seja é um ponto de partida para se basear ao dizer o quão eficiente é o modelo de Machine Learning.

In [4]:
from sklearn.dummy import DummyClassifier

In [5]:
model = DummyClassifier(strategy="most_frequent")
model.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [6]:
print(model.score(train_x, train_y))
print(model.score(test_x, test_y))

0.5333333333333333
0.4


Ele conseguiu um desempenho insatisfatório, como era de se esperar, de apenas 40%. Esse valor servirá de base para compararmos com o modelo SVM.

#### 3.3 SVM

Usemos agora o Suport Vector Machine

Primeiro importamos se método:

In [7]:
from sklearn.svm import SVC

Treinamos o modelo para SVC linear:

In [8]:
model = SVC(kernel="linear")
model.fit(train_x, train_y)

SVC(kernel='linear')

Vejamos o score que o modelo obteve:

In [9]:
print(f"O score nos dados de treino foi {model.score(train_x, train_y)} e nos de teste {model.score(test_x, test_y)}")

O score nos dados de treino foi 0.7777777777777778 e nos de teste 0.85


Testemos agora para polinomial:

In [10]:
model = SVC(kernel="poly")
model.fit(train_x, train_y)
print(f"O score nos dados de treino foi {model.score(train_x, train_y)} e nos de teste {model.score(test_x, test_y)}")

O score nos dados de treino foi 0.7388888888888889 e nos de teste 0.7


RBF:

In [11]:
model = SVC(kernel="rbf")
model.fit(train_x, train_y)
print(f"O score nos dados de treino foi {model.score(train_x, train_y)} e nos de teste {model.score(test_x, test_y)}")

O score nos dados de treino foi 0.5388888888888889 e nos de teste 0.4


Sigmoid:

In [12]:
model = SVC(kernel="sigmoid")
model.fit(train_x, train_y)
print(f"O score nos dados de treino foi {model.score(train_x, train_y)} e nos de teste {model.score(test_x, test_y)}")

O score nos dados de treino foi 0.5333333333333333 e nos de teste 0.4


### 4. Discussão
O modelo linear apresentou acurácia significativamente maior em comparação aos demais. Enquanto o baseline conseguiu 0.4, o rbf e sigmoid também obtiveram 0.4, o polinomial obteve 0.7 e o linear 0.85. Isso além de mostrar como o mais simples pode ser o mais eficaz, mostra que cada modelo se comporta melhor em cada tipo de dataframe. O Support Vector Machine no modelo linear conseguiu uma acurácia bem maior em comparação com o baseline, de 0.4 para 0.85, mostrando grande eficiência do algoritmo em prever o resultado.

### 5. Conclusão
Portanto, o SVM (Support Vector Machine) se provou um modelo de Machine Learning interessante e útil, podendo chegar a ótimos resultados com facilidade.

### 6. Referências

PEDREGOSA, F. et al. 1.4. Support Vector Machines - Mathematical formulation. Scikit-learn, 2025. Disponível em: https://scikit-learn.org/stable/modules/svm.html#svm-mathematical-formulation. Acesso em: 29 set. 2025.

PEDREGOSA, F. et al. 1.4. Support Vector Machines. Scikit-learn, 2025. Disponível em: https://scikit-learn.org/stable/modules/svm.html. Acesso em: 29 set. 2025.

KRISH NAIK. Support Vector Machine (SVM) Intuition- Machine Learning. YouTube, 29 set. 2025. Disponível em: https://www.youtube.com/watch?v=T9UcK-TxQGw. Acesso em: 29 set. 2025.

RAJ, R. K. Support Vector Machines (SVM) - Machine Learning Tutorial. YouTube, 29 set. 2025. Disponível em: https://www.youtube.com/watch?v=FB5EdxAGxQg&list=PLeo1K3hjS3uvCeTYTeyfe0-rN5r8zn9rw&index=11. Acesso em: 29 set. 2025.

GEEKSFORGEEKS. Major Kernel Functions in Support Vector Machine (SVM). Machine Learning, 2025. Disponível em: https://www.geeksforgeeks.org/machine-learning/major-kernel-functions-in-support-vector-machine-svm/. Acesso em: 29 set. 2025.


GEEKSFORGEEKS. Support Vector Machine Algorithm. Machine Learning, 2025. Disponível em: https://www.geeksforgeeks.org/machine-learning/support-vector-machine-algorithm/. Acesso em: 29 set. 2025.

Deep Seek. Disponível em: https://chat.deepseek.com/share/fhe45hq69byce0tfx3. Acesso em: 29 set. 2025.